In [ ]:
import os
import sys

import tensorflow as tf

tf.get_logger().setLevel('ERROR')

In [ ]:
def create_model_config():
    CONFIG_DIR = os.path.join(os.getcwd(), "configs/")
    CONFIG_FILE = os.path.join(CONFIG_DIR, "default.yaml")
    if not os.path.exists(CONFIG_DIR):
        os.mkdir(CONFIG_DIR)

    config_text = \
    """image_size: 640x480
    num_classes: 1
    label_map: {1: 1d}
    """

    with open(CONFIG_FILE, "w") as fwrite:
        fwrite.write(config_text)

In [ ]:
def cleanup_model_output_dir():
    if os.path.exists(MODEL_DIR_TMP):
        !rm -rf {MODEL_DIR_TMP}

In [ ]:
def retrain_efficientdet_for_barcode(model, tfrecords_dir):
    print(tf.version.VERSION)
    create_model_config()

    MODEL = model
    CKPT = MODEL
    TFRECORDS_DIR = tfrecords_dir

    TRAIN_SET = os.path.join(TFRECORDS_DIR, "train.tfrecord-*")
    VAL_SET = os.path.join(TFRECORDS_DIR, "val.tfrecord-*")
    MODEL_DIR_TMP = os.path.join(os.getcwd(), "tmp", f"{MODEL}-finetune")
    TRAIN_NUM_EXAMPLES = 10
    EVAL_NUM_EXAMPLES = 10
    EPOCHS = 4
    BATCH_SIZE = 2    
    cleanup_model_output_dir()
    
    EFFICIENTDET_MAIN = os.path.join(os.getcwd(), 'automl/efficientdet/main.py')
    !python {EFFICIENTDET_MAIN} \
        --mode=train_and_eval \
        --train_file_pattern={TRAIN_SET} \
        --val_file_pattern={VAL_SET} \
        --model_name={MODEL} \
        --model_dir={MODEL_DIR_TMP} \
        --ckpt={CKPT} \
        --train_batch_size={BATCH_SIZE} \
        --eval_batch_size={BATCH_SIZE} \
        --num_epochs={EPOCHS} \
        --num_examples_per_epoch={TRAIN_NUM_EXAMPLES} \
        --eval_samples={EVAL_NUM_EXAMPLES} \
        --hparams={CONFIG_FILE} \
        --strategy=gpus
    
    return MODEL_DIR_TMP